# Generalizing a Taylor Recurrence

In [1]:
from sumpy.recurrence import _make_sympy_vec

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_taylor_expression, get_off_axis_recurrence

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [2]:
var = _make_sympy_vec("x", 2)
s = sp.Function("s")
n = sp.symbols("n")
i = sp.symbols("i")

In [3]:
from sumpy.expansion.diff_op import laplacian,make_identity_diff_op
w = make_identity_diff_op(2)
laplace2d = laplacian(w)

In [4]:
get_off_axis_recurrence(laplace2d)[2].subs(n, 3)

(-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2


2*s(1)/x1**2

In [5]:
w = make_identity_diff_op(2)
helmholtz2d = laplacian(w) + w
get_off_axis_recurrence(helmholtz2d)

(-6*n + (n + 1)**2 + 2)*s(n - 4)/x1**2 + (-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2


(3,
 4,
 (-6*n + (n + 1)**2 + 2)*s(n - 4)/x1**2 + (-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2)

In [6]:
get_off_axis_recurrence(helmholtz2d)[2].subs(n, 4)

(-6*n + (n + 1)**2 + 2)*s(n - 4)/x1**2 + (-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2


3*s(0)/x1**2 + 6*s(2)/x1**2

In [9]:
get_taylor_expression(laplace2d, 0)

(-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2


x0**3*s(1)/(3*x1**2) + 2*s(-2)/x1**2